# Background

Our team is collecting data from customers' router setups and storing them in a table titled data_analyst_assessment. At each screen of the setup process, a row is created in this table with a record of the setup step, along with various metrics. These metrics are: 

* Setup_id – each time a router is setup, all events tied to that setup are given this unique
ID
* Platform – the platform through which the router was setup
* Type – a category for the event
* Timestamp – the time of the event
* Model – undefined
* Step – the specific page in a multi-page setup process
* Duration – the elapsed time of the setup process at that event
* Rating – a self-reported 1-5 star rating of the setup process

# Objective

The purpose of this report is to identify the steps of the router setup process where customers are struggling the most, so that the team can plan and set priorities accordingly.

# Walk-Through of Methodology & Results

First, let's import any necessary libraries and set the parameters for viewing and plotting dataframes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# set layout parameters 
pd.set_option('precision', 3)
plt.style.use('fivethirtyeight')


Now, let's read in the dataframe and take a look at some of the rows.

In [ ]:
df = pd.read_csv('/Users/dmopo/Downloads/data_analyst_assessment.csv')


In [ ]:
df.head()


In [ ]:
df.tail()


Right off the bat, we can make some observations about the data: 

* Platform – we see 'iOS' values, as well as some nulls
* Type – we see 'setup_step' and 'setup_error'
* Timestamp – the setups occurred during the month of August
* Model – we see only one value, 'router'
* Step – we see 'selected_account' and 'setup_start'
* Duration – values up to 1417.937, this is likely in seconds 
* Rating – all null

Let's display all of the values for each column, as well as the counts for each value. We'll ignore the 'setup_id', 'timestamp', and 'duration' columns, so that we're only looking at the categorical variables in the dataset.

In [ ]:
cols = list(df.columns)
cols.remove('setup_id')
cols.remove('timestamp')
cols.remove('duration')

for col in cols:
    value_counts = df[col].value_counts()
    value_counts_df = pd.DataFrame(value_counts)
    print(value_counts_df)
    

We see some values here that didn't show up in the previous view of the dataframe. Notably:

* Platform – we see 'Android' now as well
* Type – there are 3 values in addition to the ones we saw before 
* Model – 'router' is the only value
* Step – a host of values denoting the step in the setup process
* Rating – 86 non-null values

Let's start digging a little deeper into the data.

### Duration Analysis

We can use duration of a step as a proxy for how challenging that step was for a customer. If a customer is staying on a screen for a while, they likely are struggling with completing the tasks asked of them on that screen.

Let's create a new dataframe that contains no null values for the 'duration' column. We can then make a boxplot of the duration values to get an idea of the distribution.

In [ ]:
duration_df = df[df['duration'].notnull()]
print(duration_df.shape)
plt.boxplot(duration_df['duration'])
plt.title('Step Duration')
plt.show()


In [ ]:
1e12 / 3600 / 24 / 365

It looks like there's an outlier at over 1e12. If our assumption that this column has units of seconds is correct, this amounts to over 30,000 years. This datapoint is clearly an anomaly. Let's take a look at the row containing this point.

In [ ]:
df[df['duration'] >= 1e12]


It turns out that the circle at that high outlier value is actually a cluster of circles, representing multiple datapoints, which explains why the circle border is so thick. All of the records with a ridiculously high value for 'duration' have a duration of approximately 1.6e12. Futhermore, all of these records have 'setup_error' as the step type. Let's remove these records from our dataframe and try the boxplot again.

In [ ]:
duration_df = df[df['duration'].notnull() & (df['type'] != 'setup_error')]
print(duration_df.shape)
plt.boxplot(duration_df['duration'])
plt.show()


In [ ]:
5e6 / 3600 / 24 

We have another outlier here, with a duration at around 5 million, which is approximately 58 days. This is likely another outlier. Let's take a look at the record containing this value.

In [ ]:
duration_df[duration_df['duration'] >= 1000000]


Nothing in particular jumps out from this table. Let's remove the outlier from our dataframe and give the boxplot another try.

In [ ]:
duration_df = duration_df[duration_df['duration'] < 1000000]
print(duration_df.shape)
plt.boxplot(duration_df['duration'])
plt.show()


In [ ]:
50000 / 3600

That looks better. We can get a better idea of the distribution. The majority of values are rather small, but the distribution has a long tail, with a few values making their way into the tens of thousands. This is feasible, as the largest value, at around 50 thousand, would be approximately 13 hours. 

Let's take a look at some metrics that will give us another view of the distribution -- mean, minimum, median, maximum -- grouping by setup step and sorting by mean duration.

In [ ]:
duration_df.groupby('step').agg(
    {'duration': ['count', 'mean', 'min', 'median', 'max']}
).sort_values(('duration', 'mean'), ascending = False)


We can see that the steps with the highest mean duration are the 'advanced_setup', 'no_internet', and 'controller_survey' steps, which it turns out are also the steps with the highest maximum duration. This makes sense, as we would expect advanced setup to be, as the name suggests, more advanced, and thus more difficult to achieve; no internet is also an issue that may require more time and pose more difficulties to resolve. Controller Survey sounds like a survey, probably when the customer gives their rating -- thus the long duration is probably due to the fact that a survey takes some time, rather than due to customers having issues. 

Now, let's look at another proxy for how challenging a step is: rating.

### Rating Analysis

Let's create a new dataframe, similar to the prior one, that contains no null values for the 'rating' column, and then let's plot it using a boxplot.

In [ ]:
rating_df = df[df['rating'].notnull()]
print(rating_df.shape)
plt.boxplot(rating_df['rating'])
plt.show()


It looks like most of the ratings are at 5, but there is at least one of each rating value from 1-5.

Let's look at some aggregate measures of rating by step.

In [ ]:
rating_df.groupby('step').agg(
    {'rating': ['count', 'mean', 'min', 'median', 'max']})


We can see here that the majority of the ratings are for the 'controller_survey' step. This corroborates the hypothesis made earlier that 'controller_survey' is when the customer provides their rating. Due to this hypothesis, as well as the fact that there isn't much data in any of the other steps, using rating as a proxy for level of difficulty of a step seems to be inconclusive. However, there is one other proxy we can look at.

### Error Analysis

We can use errors as a sign that a customer is struggling with a step. Let's create a pivot table of type versus step. Since one of the types is 'setup_error', we can take a look at which steps have both the highest raw count as well as the highest percentage of errors. 

In [ ]:
error_df = df.groupby(['step', 'type']).size().unstack(fill_value=0)
error_df['error_perc'] = round(error_df['setup_error'] / 
                               (error_df['setup_step'] + error_df['setup_rating'] + error_df['setup_error'])
                               , 3)
error_df.sort_values(['error_perc', 'setup_error'], ascending = False)


It looks like the only step with a significant percentage of errors is 'setting_up', with over half of the step instances being errrors. The step with the next highest error percentage is 'setup_start', at only 8%; this step can be lumped in with setup. Only two of the other steps have any errors, and their percentages are so low they are negligible. The errors for the 'credentials' step could possibly be from entering a password incorrectly. 

## Conclusion

In conclusion, the areas we recommend the team focus on are advanced setup and dealing with no internet, due to the long durations of these steps on average, as well as actually setting up the router, due to the high error rate. 